In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os
FOLDER_NAME = 'dataset_extracted'

CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


'2.10.0'

In [3]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in CLASSES:
            label_map[foldername] = CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [4]:
sequence, label = [], []
target_length = 48
for (root, folders, files) in os.walk(FOLDER_NAME):
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if(os.path.split(file_path)[0] in CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root)])

print(np.array(sequence).shape)
print(np.array(label).shape)


(1690, 48, 1662)
(1690,)


In [5]:
# FOR DATA DUPLICATION
# sequence = np.concatenate([sequence, sequence, sequence, ], axis=0)
# label = np.concatenate([label, label, label], axis=0)

# print(np.array(sequence).shape)
# print(np.array(label).shape)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(CLASSES).shape[0], dtype='float32'), test_size=0.05)

print(X_train.shape, X_test.shape)


(1605, 48, 1662) (85, 48, 1662)


In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard


In [8]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [15]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(48, 1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(CLASSES).shape[0], activation='softmax'))


In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [36]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


In [24]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 48, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 48, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 26)                858       
                                                                 
Total params: 597,434
Trainable params: 597,434
Non-tr

In [25]:
res = model.predict(X_test)

3/3 [==============================] - 0s 42ms/step
